In [0]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
from skimage import transform
import torchvision.transforms as transforms
from torch.autograd import Variable
import pandas as pd;
import numpy as np;
from torch.utils.data import Dataset, DataLoader
import random;
import math;
import torch.nn.functional as F
import torchvision
import matplotlib.pyplot as plt
import torch.optim as optim

In [3]:
train_set = torchvision .datasets. FashionMNIST (root = ".", train = True ,
download = True , transform = transforms .ToTensor ())
test_set = torchvision .datasets. FashionMNIST (root = ".", train = False ,
download = True , transform = transforms .ToTensor ())
training_loader = torch.utils.data. DataLoader (train_set , batch_size = 32,
shuffle = False)
test_loader = torch.utils.data. DataLoader (test_set , batch_size = 32,
shuffle = False)
torch. manual_seed (0)
# If you are using CuDNN , otherwise you can just ignore
torch.backends.cudnn. deterministic = True
torch.backends.cudnn. benchmark = False


Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw



Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw
Processing...
Done!


In [4]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        
        # Convolution 1
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=5, stride=1, padding=0)
        self.relu1 = nn.ReLU()
        # Max pool 1
        self.maxpool1 = nn.MaxPool2d(kernel_size=2)
     
        # Convolution 2
        self.cnn2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=0)
        self.relu2 = nn.ReLU()

        #self.dropout = nn.Dropout(p=1)

        # Max pool 2
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        # Fully connected  (readout)

        #self.fc_model = nn.Sequential(nn.Linear(64*4*4, 256), nn.Tanh(), nn.Linear(256,10))
        self.fc1 = nn.Linear(in_features=1024, out_features=256)
        self.relu3 = nn.ReLU()
        self.dropout = nn.Dropout(p=0.01)
        self.fc2 = nn.Linear(in_features=256, out_features=10)

    
    def forward(self, x):
        # Convolution 1
        out = self.cnn1(x)
        out = self.relu1(out)
        
        # Max pool 1
        out = self.maxpool1(out)
        
        # Convolution 2 
        out = self.cnn2(out)
        out = self.relu2(out)

        #out = self.dropout(out)
        
        # Max pool 2 
        out = self.maxpool2(out)
        

        out = out.view(out.size(0), -1)
        # x = self.fc_model(x)
        out = self.fc1(out)
        out = self.relu3(out)
        out = self.dropout(out)
        out = self.fc2(out)
        
        return out

In [0]:
def evaluation(dataloader):
  total, correct = 0,0
  net.eval()
  for data in dataloader:
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    outputs = net(inputs)
    _, pred = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (pred == labels).sum().item()
  return 100 * correct / total


In [0]:
def weights_init(layer):
    if isinstance(layer, nn.Linear):
      nn.init.xavier_uniform_(layer.weight)


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

net = CNNModel().to(device)
net.apply(weights_init)
loss_fn = nn.CrossEntropyLoss()
# SGD optimizer with learning rate of 0.1
opt = torch.optim.SGD(list(net.parameters()), lr = 0.1)



# Training phase

# Change this value to make training longer
max_epochs = 50
loss_epoch_array = []
loss_epoch = 0
train_accuracy = []
valid_accuracy = []
for epoch in range(max_epochs):
  loss_epoch = 0
  for i, data in enumerate(training_loader, 0):
    net.train()
    inputs, labels = data
    inputs, labels = inputs.to(device), labels.to(device)
    opt.zero_grad()
    outputs = net(inputs)
    loss = loss_fn(outputs, labels)
    loss.backward()
    opt.step()
    loss_epoch += loss.item()
  loss_epoch_array.append(loss_epoch)
  train_accuracy.append(evaluation(training_loader))
  valid_accuracy.append(evaluation(test_loader))
  print("Epoch {}: loss: {}, train accuracy: {}, valid accuracy:{}".format(epoch + 1, loss_epoch_array[-1], train_accuracy[-1], valid_accuracy[-1]))

Epoch 1: loss: 925.4339983388782, train accuracy: 88.02, valid accuracy:86.87
Epoch 2: loss: 593.3765522018075, train accuracy: 90.35166666666667, valid accuracy:88.61
Epoch 3: loss: 505.72572898492217, train accuracy: 91.37333333333333, valid accuracy:89.55
Epoch 4: loss: 448.61480568908155, train accuracy: 92.05333333333333, valid accuracy:90.04
Epoch 5: loss: 402.10969565063715, train accuracy: 92.62666666666667, valid accuracy:89.96
Epoch 6: loss: 360.0619649300352, train accuracy: 93.22166666666666, valid accuracy:90.09
Epoch 7: loss: 322.92799665406346, train accuracy: 93.01333333333334, valid accuracy:89.96
Epoch 8: loss: 290.5694156270474, train accuracy: 94.12, valid accuracy:90.67


In [0]:
plt.plot(train_accuracy, label = "Training accuracy")
plt.plot(valid_accuracy, label = "Validation accuracy")
plt.legend(frameon = False)